In [ ]:
!cd python-package/
!pip install -v .

/bin/bash: line 1: cd: python-package/: No such file or directory
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
import keras
print(keras.__version__)

2.15.0


In [ ]:
import os
os.environ["KERAS_BACKEND"] = "jax"
import keras

#Un guide complet pour comprendre et implémenter la classification de texte en Python

In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string

## 1. Préparation de l'ensemble de données

Pour les besoins de cet article, j'utilise un ensemble de données d'avis Amazon qui peuvent être téléchargés sur ce lien . L'ensemble de données se compose de 3,6 millions de critiques de textes et de leurs étiquettes, nous n'utiliserons qu'une petite fraction des données. Pour préparer l'ensemble de données, chargez les données téléchargées dans un dataframe pandas contenant deux colonnes : texte et étiquette. ( Source )

In [ ]:
# load the dataset
data = open('/content/corpus').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))

# create a dataframe using texts and lables
trainDF = pandas.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

Ensuite, nous diviserons l'ensemble de données en ensembles de formation et de validation afin de pouvoir former et tester le classificateur. Nous coderons également notre colonne cible afin qu’elle puisse être utilisée dans des modèles d’apprentissage automatique.

In [ ]:
# split the dataset into training and validation datasets
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# label encode the target variable
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
encoder

LabelEncoder()

In [ ]:
train_y

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
valid_y

array([1, 1, 1, ..., 1, 0, 1])

##2. Ingénierie des fonctionnalités
La prochaine étape est l’étape d’ingénierie des fonctionnalités. Au cours de cette étape, les données textuelles brutes seront transformées en vecteurs de caractéristiques et de nouvelles fonctionnalités seront créées à l'aide de l'ensemble de données existant. Nous mettrons en œuvre les différentes idées suivantes afin d'obtenir des fonctionnalités pertinentes à partir de notre ensemble de données.

2.1 Compter les vecteurs comme caractéristiques

2.2 Vecteurs TF-IDF comme caractéristiques

Niveau de mot

Niveau N-Gram

Niveau du personnage

2.3 Intégrations de mots en tant que fonctionnalités

2.4 Fonctionnalités basées sur le texte / PNL

2.5 Modèles de sujets en tant que fonctionnalités

Examinons en détail la mise en œuvre de ces idées.

**2.1 Compter les vecteurs comme caractéristiques**

Count Vector est une notation matricielle de l'ensemble de données dans laquelle chaque ligne représente un document du corpus, chaque colonne représente un terme du corpus et chaque cellule représente le nombre de fréquences d'un terme particulier dans un document particulier.

In [ ]:
# create a count vectorizer object
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
xtrain_count

<7500x31666 sparse matrix of type '<class 'numpy.int64'>'
	with 437415 stored elements in Compressed Sparse Row format>

**2.2 Vecteurs TF-IDF en tant que caractéristiques**

Le score TF-IDF représente l'importance relative d'un terme dans le document et dans l'ensemble du corpus. Le score TF-IDF est composé de deux termes : le premier calcule la fréquence des termes normalisée (TF), le deuxième terme est la fréquence inverse des documents (IDF), calculée comme le logarithme du nombre de documents dans le corpus divisé par le nombre. des documents où le terme spécifique apparaît.

TF(t) = (Nombre de fois où le terme t apparaît dans un document) / (Nombre total de termes dans le document)

IDF(t) = log_e (Nombre total de documents / Nombre de documents contenant le terme t)

Les vecteurs TF-IDF peuvent être générés à différents niveaux de jetons d'entrée (mots, caractères, n-grammes)

**a. Word Level TF-IDF:** Matrice représentant les scores tf-idf de chaque terme dans différents documents

**b. N-gramme Niveau TF-IDF:**  Les N-grammes sont la combinaison de N termes ensemble. Cette matrice représentant les scores tf-idf de N-grammes

**c. Character Level TF-IDF:** Matrice représentant les scores tf-idf des n-grammes de niveau caractère dans le corpus

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x)
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:558: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn(


In [ ]:
tfidf_vect_ngram_chars

TfidfVectorizer(analyzer='char', max_features=5000, ngram_range=(2, 3),
                token_pattern='\\w{1,}')

**2.3 Word Embeddings === Intégrations de mots**

Une intégration de mots est une forme de représentation de mots et de documents à l'aide d'une représentation vectorielle dense. La position d'un mot dans l'espace vectoriel est apprise à partir du texte et est basée sur les mots qui entourent le mot lorsqu'il est utilisé. Les incorporations de mots peuvent être entraînées à l'aide du corpus d'entrée lui-même ou peuvent être générées à l'aide d'incorporations de mots pré-entraînées telles que Glove, FastText et Word2Vec . N’importe lequel d’entre eux peut être téléchargé et utilisé comme apprentissage par transfert. On peut en savoir plus sur les intégrations de mots  ici.

L'extrait suivant montre comment utiliser les intégrations de mots pré-entraînées dans le modèle. Il y a quatre étapes essentielles :

Chargement des intégrations de mots pré-entraînées

Création d'un objet tokenizer

Transformer des documents texte en séquence de jetons et les compléter

Créer un mappage des jetons et de leurs intégrations respectives

In [ ]:
# load the pre-trained word-embedding vectors
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

**2.4 Fonctionnalités basées sur le texte/PNL**

Un certain nombre de fonctionnalités supplémentaires basées sur le texte peuvent également être créées, qui sont parfois utiles pour améliorer les modèles de classification de texte. Certains exemples sont:

Nombre de mots des documents – nombre total de mots dans les documents
Nombre de caractères des documents – nombre total de caractères dans les documents

Densité moyenne des mots des documents – longueur moyenne des mots utilisés dans les documents

Nombre de ponctuations dans l'essai complet – nombre total de signes de ponctuation dans les documents
Nombre de majuscules dans l'essai complet - nombre total de mots en majuscules dans les documents

Nombre de mots du titre dans l'essai complet - nombre total de mots en casse (titre) dans les documents

Distribution de fréquence des balises de partie du discours :

Nombre de noms

Nombre de verbes

Nombre d'adjectifs

Nombre d'adverbes

Nombre de pronoms

Ces fonctionnalités sont hautement expérimentales et doivent être utilisées uniquement en fonction de l'énoncé du problème.

In [ ]:
trainDF [ 'char_count' ]  =  trainDF [ 'text' ] . apply ( len )
trainDF [ 'word_count' ]  =  trainDF [ 'text' ] . appliquer ( lambda  x :  len ( x . split ()))
trainDF [ 'word_density' ]  =  trainDF [ 'char_count' ]  /  ( trainDF [ 'word_count' ] + 1 )
trainDF [ 'punctuation_count' ]  =  trainDF [ 'text' ] . apply ( lambda  x :  len ( "" . join ( _  for  _  in  x  if  _  in  string . ponctuation )))
trainDF [ 'title_word_count' ]  =  trainDF [ 'text' ] . apply ( lambda  x :  len ([ wrd  for  wrd  in  x . split ()  if  wrd . istitle ()]))
trainDF [ 'upper_case_word_count' ]  =  trainDF [ 'text' ] . appliquer ( lambda  x :  len ([ wrd  for  wrd  in  x . split ()  if  wrd . isupper ()]))

In [ ]:
# fonction pour vérifier et obtenir le nombre de balises de partie du discours d'un mot dans une phrase donnée
def  check_pos_tag ( x ,  flag ) :
    cnt  =  0
    try :
        wiki  =  textblob . TextBlob ( x )
        pour  tup  dans  wiki . tags :
            ppo  =  list ( tup )[ 1 ]
            si  ppo  dans  pos_family [ flag ] :
                cnt  +=  1
    sauf :
        pass
    return  cnt

**3. Construction de modèles**

La dernière étape du cadre de classification de texte consiste à former un classificateur à l'aide des fonctionnalités créées à l'étape précédente. Il existe de nombreux choix différents de modèles d'apprentissage automatique qui peuvent être utilisés pour entraîner un modèle final. Nous implémenterons les différents classificateurs suivants à cet effet :

Classificateur naïf de Bayes

Classificateur linéaire

Machine à vecteurs de support

Modèles d'ensachage

Stimuler les modèles

Réseaux de neurones peu profonds

Réseaux de neurones profonds

Réseau neuronal convolutif (CNN)

Modélisateur Long Court Terme (LSTM)

Unité récurrente fermée (GRU)

RNN bidirectionnel

Réseau neuronal convolutif récurrent (RCNN)

**Autres variantes de réseaux de neurones profonds**

Implémentons ces modèles et comprenons leurs détails. La fonction suivante est une fonction utilitaire qui peut être utilisée pour entraîner un modèle. Il accepte le classificateur, feature_vector des données d'entraînement, les étiquettes des données d'entraînement et les vecteurs de caractéristiques des données valides comme entrées. À l'aide de ces entrées, le modèle est entraîné et le score de précision est calculé.

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)

    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)

    if is_neural_net:
        predictions = predictions.argmax(axis=-1)

    return metrics.accuracy_score(predictions, valid_y)

**3.1 Bayes naïfs**

Implémentation d'un modèle bayésien naïf à l'aide de l'implémentation sklearn avec différentes fonctionnalités

Naive Bayes est une technique de classification basée sur le théorème de Bayes avec une hypothèse d'indépendance entre les prédicteurs. Un classificateur Naive Bayes suppose que la présence d'une fonctionnalité particulière dans une classe n'est pas liée à la présence d'une autre fonctionnalité

In [ ]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print ("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print( "NB, CharLevel Vectors: ", accuracy)

**3.2 Classificateur linéaire**

Implémentation d'un classificateur linéaire (régression logistique)

La régression logistique mesure la relation entre la variable dépendante catégorielle et une ou plusieurs variables indépendantes en estimant les probabilités à l'aide d'une fonction logistique/sigmoïde. On peut en savoir plus sur la régression logistique

In [ ]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print( "LR, CharLevel Vectors: ", accuracy)

**3.3 Implémentation d'un modèle SVM**

Support Vector Machine (SVM) est un algorithme d'apprentissage automatique supervisé qui peut être utilisé à la fois pour des défis de classification ou de régression. Le modèle extrait le meilleur hyperplan/ligne possible qui sépare les deux classes

In [ ]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, N-Gram Vectors: ", accuracy)

**3.4 Modèle d'ensachage = Bagging Model**

Implémentation d'un modèle de forêt aléatoire

Les modèles Random Forest sont un type de modèles d’ensemble, en particulier les modèles d’ensachage. Ils font partie de la famille des modèles arborescents. On peut en savoir plus sur l'ensachage et les forêts aléatoires

In [ ]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print ("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("RF, WordLevel TF-IDF: ", accuracy)

**3.5 Modèle de renforcement = Boosting Model**

Implémentation du modèle d'amplification de gradient Xtereme

Les modèles de boosting sont un autre type de modèles d’ensemble faisant partie des modèles arborescents. Boosting est un méta-algorithme d'ensemble d'apprentissage automatique destiné principalement à réduire les biais, ainsi que la variance dans l'apprentissage supervisé, et une famille d'algorithmes d'apprentissage automatique qui convertissent les apprenants faibles en apprenants forts. Un apprenant faible est défini comme un classificateur qui n'est que légèrement corrélé à la véritable classification (il peut mieux étiqueter les exemples que les devinettes aléatoires). En savoir plus sur ces modèles

In [ ]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print( "Xgb, Count Vectors: ", accuracy)

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print ("Xgb, WordLevel TF-IDF: ", accuracy)

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print ("Xgb, CharLevel Vectors: ", accuracy)

**3.6 Réseaux de neurones peu profonds**

Un réseau de neurones est un modèle mathématique conçu pour se comporter de manière similaire aux neurones biologiques et au système nerveux. Ces modèles sont utilisés pour reconnaître des modèles et des relations complexes qui existent au sein de données étiquetées. Un réseau de neurones peu profond contient principalement trois types de couches : la couche d'entrée, la couche cachée et la couche de sortie. En savoir plus sur les réseaux de neurones

In [ ]:
def create_model_architecture(input_size):
    # create input layer
    input_layer = layers.Input((input_size, ), sparse=True)

    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)

    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True)
print ("NN, Ngram Level TF IDF Vectors",  accuracy)

**3.7 Réseaux de neurones profonds**

Les réseaux de neurones profonds sont des réseaux de neurones plus complexes dans lesquels les couches cachées effectuent des opérations beaucoup plus complexes que de simples activations sigmoïdes ou relu. Différents types de modèles d’apprentissage profond peuvent être appliqués aux problèmes de classification de textes.



*3.7.1 Réseau neuronal convolutif*

Dans les réseaux de neurones convolutifs, les convolutions sur la couche d'entrée sont utilisées pour calculer la sortie. Il en résulte des connexions locales, où chaque région de l'entrée est connectée à un neurone de la sortie. Chaque couche applique différents filtres et combine leurs résultats.

In [ ]:
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')

    return model

classifier = create_cnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("CNN, Word Embeddings",  accuracy)

*3.7.2 Réseau neuronal récurrent – ​​LSTM*

Contrairement aux réseaux de neurones Feed-forward dans lesquels les sorties d'activation se propagent dans une seule direction, les sorties d'activation des neurones se propagent dans les deux sens (des entrées aux sorties et des sorties aux entrées) dans les réseaux de neurones récurrents. Cela crée des boucles dans l'architecture du réseau neuronal qui agit comme un « état mémoire » des neurones. Cet état permet aux neurones de se souvenir de ce qui a été appris jusqu'à présent.

L'état de la mémoire dans les RNN donne un avantage par rapport aux réseaux de neurones traditionnels, mais un problème appelé Vanishing Gradient leur est associé. Dans ce problème, lors de l’apprentissage avec un grand nombre de couches, il devient très difficile pour le réseau d’apprendre et d’ajuster les paramètres des couches précédentes. Pour résoudre ce problème, un nouveau type de RNN appelés modèles LSTM (Long Short Term Memory) a été développé.

In [ ]:
def create_rnn_lstm():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')

    return model

classifier = create_rnn_lstm()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("RNN-LSTM, Word Embeddings",  accuracy)

*3.7.3 Réseau neuronal récurrent – ​​GRU*

Les unités récurrentes fermées sont une autre forme de réseaux neuronaux récurrents. Ajoutons une couche de GRU au lieu de LSTM dans notre réseau.

In [ ]:
def create_rnn_gru():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')

    return model

classifier = create_rnn_gru()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("RNN-GRU, Word Embeddings",  accuracy)

*3.7.4 RNN bidirectionnel*

Les couches RNN peuvent également être enveloppées dans des couches bidirectionnelles. Enveloppons notre couche GRU dans une couche bidirectionnelle.

In [ ]:
def create_bidirectional_rnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')

    return model

classifier = create_bidirectional_rnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("RNN-Bidirectional, Word Embeddings",  accuracy)

*3.7.5 Réseau neuronal convolutif récurrent*

Une fois les architectures essentielles essayées, on peut essayer différentes variantes de ces couches comme le réseau neuronal convolutif récurrent. D'autres variantes peuvent être :

Réseaux d'attention hiérarchique

Modèles séquence à séquence avec attention

Réseaux de neurones convolutifs récurrents bidirectionnels

CNN et RNN avec plus de couches

In [ ]:
def create_rcnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')

    return model

classifier = create_rcnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("CNN, Word Embeddings",  accuracy)

##Améliorer les modèles de classification de texte

Bien que le cadre ci-dessus puisse être appliqué à un certain nombre de problèmes de classification de texte, pour obtenir une bonne précision, certaines améliorations peuvent être apportées au cadre global. Par exemple, voici quelques conseils pour améliorer les performances des modèles de classification de texte et de ce cadre.

1. Nettoyage de texte :  le nettoyage de texte peut aider à réduire le bruit présent dans les données textuelles sous forme de mots vides, de signes de ponctuation, de variations de suffixes, etc. Cet  article  peut aider à comprendre comment implémenter la classification de texte en détail.

2. Fonctionnalités Hstacking Text / NLP avec des vecteurs de fonctionnalités de texte :  dans la section d'ingénierie des fonctionnalités, nous avons généré un certain nombre de vecteurs de fonctionnalités différents, leur combinaison peut aider à améliorer la précision du classificateur.

3. Réglage des hyperparamètres dans la modélisation :  Le réglage des paramètres est une étape importante, un certain nombre de paramètres tels que la longueur de l'arbre, les feuilles, les paramètres de réseau, etc. peuvent être ajustés pour obtenir le modèle le mieux adapté.

4. Modèles d'ensemble :  empiler différents modèles et mélanger leurs sorties peut aider à améliorer encore les résultats.